In [ ]:
!pip install -U bitsandbytes transformers accelerate \
    langchain langchain-community langchain-huggingface \
    chromadb openpyxl requests beautifulsoup4 PyPDF2

In [ ]:
'''
import re

with open("bio_urls.txt", "r") as f:
    cleaned_urls = [re.sub(r'(?<!:)//+', '/', line.strip()) for line in f if line.strip()]

with open("bio_urls_cleaned.txt", "w") as f:
    for url in cleaned_urls:
        f.write(url + "\n")
'''

In [ ]:
'''
import requests
from langchain.document_loaders import PyPDFLoader
from io import BytesIO

def fetch_and_load_pdf(url: str):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }
    try:
        print(f"🔗 Trying URL: {url}")
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Check for correct content type (application/pdf)
        if "application/pdf" not in response.headers.get("Content-Type", ""):
            print(f"❌ Not a PDF (content-type was: {response.headers.get('Content-Type')})")
            return None

        loader = PyPDFLoader(BytesIO(response.content))
        return loader.load()
    except Exception as e:
        print(f"❌ Exception while fetching PDF from {url}: {e}")
        return None
'''

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import openpyxl
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import pandas as pd

# ===== MODEL SETUP =====
model_name = "aaditya/Llama3-OpenBioLLM-8B"

quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.float16
)

def run_biollm(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
                  **inputs,
                  max_new_tokens=256,
                  do_sample=True,
                  pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):].strip()

# ===== GLOBAL SETUP =====
curRow = 2
columnHeaders = ["MODEL", "Prompt", "Question", "Context", "ContextSource", "Answer"]
headers = {header: idx + 1 for idx, header in enumerate(columnHeaders)}

# ===== UTILITIES =====
def write_cell(sheet, row, col, value):
    sheet.cell(row=row, column=col).value = value

def add_row(sheet, row_data):
    global curRow
    for key in columnHeaders:
        sheet.cell(row=curRow, column=headers[key]).value = row_data.get(key, "")
    curRow += 1

def load_pdf_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        with open("temp.pdf", "wb") as f:
            f.write(response.content)
        reader = PdfReader("temp.pdf")
        return "".join([page.extract_text() or "" for page in reader.pages])
    except Exception as e:
        print(f"Failed to load PDF from {url}: {e}")
        return ""

def scrape_website(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        return "\n".join([p.get_text() for p in soup.find_all("p")])
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return ""

'''
def load_data():
    pdf_urls = [
        "https://www.biorxiv.org/content/10.1101/2020.07.28.224253v1.full.pdf"
    ]

    website_urls = [
        "https://en.wikipedia.org/wiki/Pathology",
        "https://www.mcgill.ca/pathology/about/definition"
    ]

    documents_with_metadata = []

    for url in pdf_urls:
        data = load_pdf_from_url(url)
        if data:
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
            splits = text_splitter.split_text(data)
            for split in splits:
                if split.strip():
                    documents_with_metadata.append(Document(page_content=split, metadata={"source": url}))

    for url in website_urls:
        data = scrape_website(url)
        if data:
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
            splits = text_splitter.split_text(data)
            for split in splits:
                if split.strip():
                    documents_with_metadata.append(Document(page_content=split, metadata={"source": url}))

    embedding_model = HuggingFaceEmbeddings(model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
    vectorstore = Chroma(
    embedding_function=embedding_model,
    persist_directory="chromadb"
    )

    vectorstore.add_documents(documents_with_metadata)
    vectorstore.persist()

    return vectorstore
'''

def load_data():
    documents_with_metadata = []

    # ========== Load from combined_health_topics_with_source.csv ==========
    try:
        df = pd.read_csv("combined_health_topics_with_source.csv")
        for _, row in df.iterrows():
            text = str(row.get("text", "")).strip()
            source = str(row.get("source", "")).strip()
            if text:
                chunks = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0).split_text(text)
                for chunk in chunks:
                    if chunk.strip():
                        documents_with_metadata.append(Document(page_content=chunk, metadata={"source": source}))
    except Exception as e:
        print(f"Error loading CSV: {e}")

    # ========== Load from textbook_pathology.txt ==========
    try:
        with open("textbook_pathology.txt", "r", encoding="utf-8") as f:
            textbook_content = f.read()
            chunks = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0).split_text(textbook_content)
            for chunk in chunks:
                if chunk.strip():
                    documents_with_metadata.append(Document(page_content=chunk, metadata={"source": "Textbook of Pathology"}))
    except Exception as e:
        print(f"Error reading textbook: {e}")

    # ===== Load BioRxiv PDF links from file =====
    try:
        with open("Usable Documents From biorxiv.org.txt", "r", encoding="utf-8") as f:
            urls = [line.strip().split("|")[0].strip() for line in f if line.strip()]
        for url in urls:
            print(f"Loading PDF from: {url}")
            text = load_pdf_from_url(url)
            if text:
                chunks = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0).split_text(text)
                for chunk in chunks:
                    if chunk.strip():
                        documents_with_metadata.append(Document(page_content=chunk, metadata={"source": url}))
    except Exception as e:
        print(f"Error loading BioRxiv URLs: {e}")

    # ========== Embed and store in Chroma ==========
    embedding_model = HuggingFaceEmbeddings(model_name="pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb")
    vectorstore = Chroma(
        embedding_function=embedding_model,
        persist_directory="chromadb"
    )
    # NEW: Batch insertion
    batch_size = 5000  # You can reduce this further if needed
    for i in range(0, len(documents_with_metadata), batch_size):
        batch = documents_with_metadata[i:i+batch_size]
        vectorstore.add_documents(batch)
    vectorstore.persist()
    return vectorstore


# ===== TWO-LAYER RAG GENERATION =====
def query_two_layer_rag(vectorstore, question):
    retriever = vectorstore.as_retriever()
    context_docs = retriever.get_relevant_documents(question)

    # First Layer: Summarize each chunk
    layer1_summaries = []
    for doc in context_docs:
        chunk = doc.page_content
        prompt_layer1 = f"""
        Summarize the following text briefly regarding this question: '{question}'.
        If irrelevant, respond with "Not relevant."

        Text: {chunk}
        """
        summary = run_biollm(prompt_layer1)
        if summary.lower() != "not relevant":
            layer1_summaries.append(summary)

    # Second Layer: Combine and summarize
    combined_summaries = "\n\n".join(layer1_summaries)
    prompt_layer2 = f"""
    Using these summaries:

    {combined_summaries}

    Generate a concise, final answer to the question '{question}'.
    """
    final_answer = run_biollm(prompt_layer2)

    return {
        "MODEL": "Llama3-OpenBioLLM-8B-Two-Layer-RAG",
        "Question": question,
        "Prompt": prompt_layer2,
        "Context": combined_summaries,
        "Answer": final_answer
    }

# ===== MAIN PIPELINE =====
def main():
    #wbq = openpyxl.load_workbook("LLM Questions.xlsx")
    wbq = openpyxl.load_workbook("100questions.xlsx")
    sheet_q = wbq.active
    questions = [sheet_q.cell(row=i, column=1).value for i in range(2, sheet_q.max_row + 1) if sheet_q.cell(row=i, column=1).value]

    vectorstore = load_data()
    wb = openpyxl.Workbook()
    sheet = wb.active
    for i, header in enumerate(columnHeaders):
        write_cell(sheet, 1, i + 1, header)

    for q in questions:
        row_data = query_two_layer_rag(vectorstore, q)
        add_row(sheet, row_data)

    wb.save("sample.xlsx")
    print("Results saved to sample.xlsx")

if __name__ == "__main__":
    main()
